In [1]:
import h5py
import tensorflow as tf
import numpy as np
import math
from matplotlib import pyplot as plt
from tensorflow.keras import layers, models, Model, Input
from tensorflow.keras.callbacks import EarlyStopping

In [49]:
hdf5_file = "energy_data.h5"

In [51]:
def hdf5_generator():
    with h5py.File(hdf5_file, 'r') as f:
        data = f['data']
        labels = f['labels']
        num_samples = data.shape[0]
        
        for i in range(num_samples):
            yield data[i], labels[i]

In [53]:
def _fixup_shape(images, labels):
    images.set_shape([32, 20, 110, 11])
    labels.set_shape([32,])
    return images, labels

In [55]:
dataset = tf.data.Dataset.from_generator(
    hdf5_generator,
    output_signature=(
        tf.TensorSpec(shape=(20, 110, 11), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int16)
    )
)
# Optimize the pipeline
dataset = (dataset
           .batch(32,drop_remainder=True)
           .map(_fixup_shape)
           .prefetch(tf.data.AUTOTUNE))
print("Data loaded")

Data loaded


In [11]:
input_3d = Input(shape=(20, 110, 11, 1), name='3D_Input')

# 3D CNN Path
x = layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu')(input_3d)  # 'same' padding to preserve dimensions
x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

x = layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

# Flatten 3D features
x = layers.Flatten()(x)

# Fully Connected Layers
fc = layers.Dense(128, activation='relu')(x)
fc = layers.Dense(64, activation='relu')(fc)

# Output Layer
output = layers.Dense(1, activation='linear')(fc)

# Define the Model
model = Model(inputs=[input_3d], outputs=output)

# Compile the Model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Model Summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 3D_Input (InputLayer)           │ (None, 20, 110, 11, 1) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 18, 108, 9, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 9, 54, 4, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 9, 54, 4, 32)   │        55,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 4, 27, 2, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       884,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 950,305 (3.63 MB)

 Trainable params: 950,305 (3.63 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
with h5py.File(hdf5_file, "r") as hdf:
    dataset_size = len(hdf["data"])  # Or hdf["labels"], if they have the same length
    print(f"Dataset size: {dataset_size}")

Dataset size: 450000


In [35]:
p=0.001
taken_size=p*dataset_size
val_dataset=dataset.take(int(0.1*dataset_size*p))
train_dataset=dataset.skip(int(0.1*p*dataset_size)).take(int(0.9*dataset_size*p))

In [37]:
early_stopping = EarlyStopping(
    monitor='val_loss',         # Metric to monitor (e.g., 'val_loss' or 'val_accuracy')
    patience=2,                 # Number of epochs with no improvement to wait before stopping
    restore_best_weights=True   # Restore model weights from the best epoch
)

In [ ]:
print("Training")
#Trening
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    shuffle=True,
    epochs=20,                 # Set a high number of epochs; EarlyStopping will stop early
    callbacks=[early_stopping],  # Add EarlyStopping to callbacks
    verbose=1
)
print("Training done")

Training
Epoch 1/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 109s 266ms/step - loss: 41.9411 - mean_absolute_error: 4.9810 - val_loss: 96.7319 - val_mean_absolute_error: 7.7583
Epoch 2/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 109s 266ms/step - loss: 41.9927 - mean_absolute_error: 4.9639 - val_loss: 72.1608 - val_mean_absolute_error: 6.5464
Epoch 3/20
241/405 ━━━━━━━━━━━━━━━━━━━━ 42s 260ms/step - loss: 39.7508 - mean_absolute_error: 4.8291

In [65]:
hdf5_file = "energy_data.h5"

In [67]:
def hdf5_generator():
    with h5py.File(hdf5_file, 'r') as f:
        data = f['data']
        labels = f['labels']
        num_samples = data.shape[0]
        
        for i in range(num_samples):
            yield data[i], labels[i]

In [69]:
def _fixup_shape(images, labels):
    images.set_shape([32, 20, 110, 11])
    labels.set_shape([32,])
    return images, labels

In [73]:
test_dataset = tf.data.Dataset.from_generator(
    hdf5_generator,
    output_signature=(
        tf.TensorSpec(shape=(20, 110, 11), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int16)
    )
)
# Optimize the pipeline
test_dataset = (test_dataset
           .batch(32,drop_remainder=True)
           .map(_fixup_shape)
           .prefetch(tf.data.AUTOTUNE))
print("Data loaded")

Data loaded


In [75]:
test_ds=test_dataset.take(int(int(0.1*dataset_size*p)))

In [77]:
test_loss, test_mae = model.evaluate(test_ds)
print("Test MAE dla sieci konwolucyjnej:", test_mae)

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - loss: 42.0869 - mean_absolute_error: 4.9855
Test MAE dla sieci konwolucyjnej: 4.908227443695068
